In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
try:
  import pyspark
except ImportError:
  !pip install pyspark
  import pyspark

import zipfile
import pandas as pd
import os

from io import TextIOWrapper
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as f
from pyspark.sql import Window
from typing import List

In [3]:
def createSession(appName: str = "PySpark") -> SparkSession:
  return SparkSession.builder.appName(appName).getOrCreate()

def createDataFrame(spark: SparkSession, path: str, file_name: str) -> DataFrame:
  with zipfile.ZipFile(f"{path}/{file_name}", "r") as zip_ref:
    with zip_ref.open(file_name.replace(".zip", ".csv")) as f:
      df_pd = pd.read_csv(TextIOWrapper(f, encoding='utf-8'))
      return spark.createDataFrame(df_pd)

In [53]:
def createFolder(creation_folder: str, folder_name: str) -> bool | str:
  try:
    os.mkdir(f"{creation_folder}/{folder_name}")
    return True, "Folder created successfully."
  except FileExistsError:
    return False, "Folder already exists."
  except Exception as e:
    return False, f"An exception ocurred: {e}"

def createFile(file_location: str, file_name: str, content: DataFrame) -> bool | str:
  try:
    (content.write
      .mode("overwrite")
      .options(header = True)
      .csv(f"{file_location}/{file_name}"))
    return True, "File created successfully."
  except Exception as e:
    return False, f"An exception ocurred: {e}"

In [54]:
def main() -> None:
  current_location: str = "/content/drive/MyDrive/Colab Notebooks/Data Engineering practice exercises/Exercise #6/data"
  file_names: List[str] = ["Divvy_Trips_2019_Q4.zip", "Divvy_Trips_2020_Q1.zip"]

  spark: SparkSession = createSession()
  df1: DataFrame = createDataFrame(spark, current_location, file_names[0])
  df2: DataFrame = createDataFrame(spark, current_location, file_names[1])

  folder_created, message = createFolder(current_location, "reports")
  print(f"Was folder created? {folder_created}\nMessage: {message}")

  # Question #1: Average trip duration per day.
  avg_trip_duration: DataFrame = (
      df1.select("tripduration")
        .agg({"tripduration": "avg"})
        .withColumnRenamed("avg(tripduration)", "avg_trip_duration")
  )
  file_created, message = createFile(f"{current_location}/reports", "avg_trip_duration.csv", avg_trip_duration)
  print(f"Was file created? {file_created}\nMessage: {message}")

  # Question #2: Trips per day.
  trips_per_day = (
      df1.withColumn("date", f.to_date(df1.start_time))
        .select("date")
        .groupBy("date")
        .count()
    )
  file_created, message = createFile(f"{current_location}/reports", "trips_per_day.csv", trips_per_day)
  print(f"Was file created? {file_created}\nMessage: {message}")

  # Question #3: Most popular trip station for each month.
  window_spec = Window.partitionBy("month").orderBy(f.col("count").desc())
  most_popular_trip_station = (
      df1.withColumn("month", f.month(df1.start_time))
        .select("month", "from_station_name")
        .groupBy("month", "from_station_name")
        .count()
        .withColumn("rank", f.row_number().over(window_spec))
        .filter(f.col("rank") == 1)
        .drop("rank")
    )
  file_created, message = createFile(f"{current_location}/reports", "most_popular_trip_station.csv", most_popular_trip_station)
  print(f"Was file created? {file_created}\nMessage: {message}")

  # Question 4: Top 3 stations each day for the last two weeks.
  window_spec = Window.partitionBy("trip_date").orderBy(f.col("trip_count").desc())

  top_stations_per_day = (
      df1.withColumn("trip_date", f.to_date(df1.start_time))
        .filter(f.col("trip_date") >= (
            df1.select(f.max("start_time")).first()[0] - f.expr("INTERVAL 14 DAYS")
        ))
        .select("trip_date", "from_station_name")
        .groupBy("trip_date", "from_station_name")
        .count()
        .withColumnsRenamed(
            {"count" : "trip_count", "from_station_name" : "station"}
        )
        .withColumn("rank", f.row_number().over(window_spec))
        .filter(f.col("rank") <= 3)
        .drop("rank")
  )
  file_created, message = createFile(f"{current_location}/reports", "top_stations_per_day.csv", top_stations_per_day)
  print(f"Was file created? {file_created}\nMessage: {message}")

  # Question #5: Who takes the longest average trips between men and women.
  avg_trip_dur_gender = (
      df1.select("gender", "tripduration")
        .groupBy("gender")
        .agg(f.avg("tripduration"))
        .withColumnsRenamed(
            {"avg(tripduration)" : "avg_trip_duration"}
        )
        .filter(f.col("gender") != "NaN")
        .orderBy("avg_trip_duration", ascending = False)
        .limit(1)
    )
  file_created, message = createFile(f"{current_location}/reports", "avg_trip_dur_gender.csv", avg_trip_dur_gender)
  print(f"Was file created? {file_created}\nMessage: {message}")

  # Question #6: Top 10 ages of those that take the longest and shortest trips.
  top_ages = (
      df1.withColumn("age", f.year(f.current_date()) - df1.birthyear)
        .select("age", "tripduration")
        .groupBy("age")
        .agg(f.avg("tripduration"))
        .withColumnsRenamed(
            {"avg(tripduration)" : "avg_trip_duration"}
        )
        .filter(f.expr("COALESCE(age, 0) > 0 AND COALESCE(avg_trip_duration, 0) > 0 AND age <> 'NaN'"))
    )
  top_10_shortest = top_ages.orderBy("avg_trip_duration", ascending = True).limit(10)
  top_10_longest = top_ages.orderBy("avg_trip_duration", ascending = False).limit(10)
  file_created, message = createFile(f"{current_location}/reports", "top_10_shortest.csv", top_10_shortest)
  print(f"Was file created? {file_created}\nMessage: {message}")
  file_created, message = createFile(f"{current_location}/reports", "top_10_longest.csv", top_10_longest)
  print(f"Was file created? {file_created}\nMessage: {message}")

if __name__ == "__main__":
  main()

Was folder created? True
Message: Folder created successfully.
Was file created? True
Message: File created successfully.
Was file created? True
Message: File created successfully.
Was file created? True
Message: File created successfully.
Was file created? True
Message: File created successfully.
Was file created? True
Message: File created successfully.
Was file created? True
Message: File created successfully.
Was file created? True
Message: File created successfully.
